# 5 eda aeda back

### 1) Set functions
- ["Okt", "Kkma", "Komoran", "Mecab", "Hannanum"]

In [1]:
# # 분할을 원하지 않는 단어 모아두기
# no_tokenized = ['#Person#', '#Person1#', '#Person2#', '#Person3#', 
# '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#사람1#',
# '#PhoneNumber#', '#Address#', '#PassportNumber#',
# '#DateOfBirth#','#SSN#','#CardNumber#','#CarNumber#','#Email#']

In [2]:
# from jamo import h2j, j2hcj

# def get_jongsung_TF(sample_text):
#     sample_text_list = list(sample_text)
#     last_word = sample_text_list[-1]
#     last_word_jamo_list = list(j2hcj(h2j(last_word)))
#     last_jamo = last_word_jamo_list[-1]

#     jongsung_TF = "T"

#     if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
#         jongsung_TF = "F"

#     return jongsung_TF
    

# # tmp 경로에 저장된 사용자 사전 명사에 새로운 단어 추가
# with open('../mecab-0.996-ko-0.9.2/mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv', "r", encoding='utf-8') as f:
#   user_dict = f.readlines()

# for word in no_tokenized:
#   jongsung_TF = get_jongsung_TF(word)
#   line = '{},*,*,*,NNP,*,{},{},*,*,*,*,*\n'.format(word, jongsung_TF, word)

#   user_dict.append(line)
  
# print(user_dict[-10:])

In [3]:
# # 재저장
# with open('../mecab-0.996-ko-0.9.2/mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv', 'w', encoding='utf-8') as f:
#   for line in user_dict:
#     f.write(line)

In [4]:
# load library
from koeda import EasyDataAugmentation, AEasierDataAugmentation
import pandas as pd
from tqdm.notebook import tqdm
import random
import time

# EDA
def augment_text_data_with_EDA(text_col, repetition):
    # 단어 바꾸기
    """입력된 문장에 대해서 EDA를 통해 데이터 증강"""
    eda = EasyDataAugmentation(
        morpheme_analyzer="Mecab"
        )
    
    random.seed(42)

    result = text_col.progress_apply(eda, p=(0.8, 0.1, 0.1, 0.05), repetition=repetition)
    print(result)
    
    return result

# AEDA
def augment_text_data_with_AEDA(text_col, repetition):
    # 단어 사이에 문자 추가하기
    """입력된 문장에 대해서 AEDA를 통해 데이터 증강"""
    aeda = AEasierDataAugmentation(
        morpheme_analyzer="Mecab",
        punctuations=[".", ",", "!", "?", ";", ":"]
    )

    random.seed(42)
    
    # result = text_col.progress_apply(aeda, p=0.05, repetition=repetition)
    result = []
    for i in tqdm(range(len(text_col))):
        time.sleep(0.01)
        a = aeda(text_col[i], p = 0.05, repetition=repetition)
        result.append(a)

    print(result)
    return result

In [5]:
# pandas tqdm 호출
tqdm.pandas() # tqdm의 pandas전용 메소드를 호출합니다.

### 2) application train data

In [6]:
# load original data
train = pd.read_csv('../data/동혁신_train.csv')

In [7]:
# eda
train2 = train.sample(n=5000, random_state=42)
train2['dialogue'] = augment_text_data_with_EDA(train2['dialogue'], 1)
train2.to_csv('../data/동혁신_mecab_eda_augmentation.csv', index = False)

  0%|          | 0/5000 [00:00<?, ?it/s]

15056    #Person1#: 보고서 작성을 시작하셨나요? #Person2#: 정보를 얻는 데...
17792    #Person1#: 잘 선택하셨습니다. 이 중국 찻잔 세트는 매가 특이하네요.#Pe...
18415    #Person1#? #전화:#. #Person2#: 주인과 통화할 수 있나요? #P...
8410     #Person1#: 오전 에이브러햄 링컨이 가장 위대한 미국인이라는 것은 법칙 풍경...
2867     #Person1#: 실례합니다. 뉴욕으로 보내는 편지의 우편 요금은 얼마인가요?#P...
                               ...                        
20481    #Person1#것 좋아요, 마크. 이제 당신 차례예요. 문가나 지난번세요 두드리에...
33417    #Person1#: 무엇을 도와드릴까요?#Person2#: 프린터를 사용움직임고 싶...
31090    #Person1#: 우리 회사에서는 새로운 직원들에게 아파트를 제공하나요?#Pers...
28864    #Person1#: 주문 도와드릴까요?#Person2#: 네, 치즈버거랑 프렌치프라...
32348    #Person1#: 이 시계는 제명 당신의 것인가요?#Person2#: 아니요, 제...
Name: dialogue, Length: 5000, dtype: object


In [8]:
# # aeda
# train3 = train.copy()
# train3['dialogue'] = augment_text_data_with_AEDA(train3['dialogue'], 1)
# train3.to_csv('../data/mecab_aeda_augmentation.csv')

In [9]:
# 자동화 안되서 분할 적용하기 위해 수동으로 적용
aeda = AEasierDataAugmentation(
    morpheme_analyzer="Mecab",
    punctuations=[".", ",", "!", "?", ";", ":"]
)
# sampling 
train_sample = train.sample(n=5000, random_state=42).reset_index(drop = True)

random.seed(42)
# result = text_col.progress_apply(aeda, p=0.05, repetition=repetition)
result = []
n = len(train_sample['dialogue'])
for i in tqdm(range(n)):
    a = aeda(train_sample['dialogue'][i], p = 0.05, repetition=1)
    result.append(a)

  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
# 2탄
for i in tqdm(range(3907, n)):
    a = aeda(train_sample['dialogue'][i], p = 0.05, repetition=1)
    result.append(a)

  0%|          | 0/1093 [00:00<?, ?it/s]

In [ ]:
# # 3탄
# for i in tqdm(range(7268, 9000)):
#     a = aeda(train_sample['dialogue'][i], p = 0.05, repetition=1)
#     result.append(a)

In [ ]:
# # 4탄
# for i in tqdm(range(19825, n)):
#     a = aeda(train_sample['dialogue'][i], p = 0.05, repetition=1)
#     result.append(a)

In [14]:
# 결과 비교
len(result)

5000

In [15]:
# 동일한 개수 확인
len(train_sample['dialogue'])

5000

In [16]:
# 저장
train3 = train_sample.copy()
train3['dialogue'] = result
train3.to_csv('../data/동혁신_mecab_aeda_augmentation.csv', index = False)

In [17]:
# 통합 후 저장
train_merge = pd.concat([train, train2, train3])
train_merge.to_csv('../data/동혁신_augment_train_merge.csv', index = False)